In [1]:
# # Requerimientos de librerías:
# !pip install spacy
# !python -m spacy download es_core_news_md

# !pip install transformers
# !pip install transformers scipy ftfy accelerate

# !git clone https://github.com/JorgeSauri/COMSINT.git 

In [2]:
from recomendaciones_comsint import Recomendador
from tqdm import tqdm

c:\Users\jsaur\anaconda3\envs\ENV1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
recomendador128 = Recomendador(fuente='recipes_traducido.csv')
recomendador128.CargarModelo(emb_size=128, version=4)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Modelo Modelos/Modelo_Nut_FV_DistilBERT_04_EMBED-128_CNN.h5 cargado con éxito.
Modelo Modelos/Modelo_Precios_FV_DistilBERT_04_EMBED-128_CNN.h5 cargado con éxito.


In [4]:
recomendador128.modeloCNN.summary()

Model: "ModeloCNNNut_128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 CapaEntrada (InputLayer)    [(None, 98304)]           0         
                                                                 
 RESHAPING (Reshape)         (None, 128, 768)          0         
                                                                 
 batch_normalization (BatchN  (None, 128, 768)         3072      
 ormalization)                                                   
                                                                 
 CONV_1 (Conv1D)             (None, 124, 512)          1966592   
                                                                 
 POOLING_1 (MaxPooling1D)    (None, 123, 512)          0         
                                                                 
 CONV_2 (Conv1D)             (None, 121, 256)          393472    
                                                  

In [5]:
df_platillos = recomendador128.FiltrarRecetario_por_CanastaBasica(similitud=0.6, verbose=True, max_rows=10)
df_platillos_recomendados2 = recomendador128.Calcular_InfoNutricional(verbose=False)
df_platillos_recomendados2

ingredientes recibidos: 
ingredientes en caché: 
Buscando recetas con ingredientes de la canasta básica... 



100%|██████████| 1090/1090 [00:15<00:00, 71.85it/s]


 

 10 platillos encontrados con similitud mayor a 0.6
Extrayendo vectores de características de los ingredientes...




100%|██████████| 10/10 [00:02<00:00,  4.88it/s]


1/1 [==============================] - 0s 214ms/step


,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo
46,Pastel de manzana de sartén de hierro de la ab...,"½ taza de mantequilla, 1 taza de azúcar morena...",0.999771,905.33,47.17,103.64,30.36,0.7
228,Alitas de pollo de mango-hábana,"30 secciones de ala de pollo, 1 (12 onzas) pue...",0.999683,1180.54,63.42,117.24,44.38,0.3
151,Pera clásica crujiente,"1 taza de avena enrollada, ? taza de azúcar mo...",0.999645,1168.70,59.99,128.41,40.78,0.3
315,"Ensalada de nectarina, tomate y pepino de verano","2 tomates, cortados en trozos, ½ pepino: pelad...",0.998769,924.76,35.87,152.20,18.98,0.3
21,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
1,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
11,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
31,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
295,Barras de granola sin avena y sin gluten (alim...,"1 ¼ tazas de almendras, divididas, 1 ¼ tazas d...",0.997911,1104.44,68.71,84.78,47.10,0.3
136,El mejor zapatero de melocotón,"1 (29 onzas) CAN Peaches en rodajas, 2 cuchara...",0.997309,1223.85,66.62,119.65,46.05,0.3


In [6]:
df2 = df_platillos_recomendados2.sort_values(by=['similitud'], ascending=False)
df2

,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo
46,Pastel de manzana de sartén de hierro de la ab...,"½ taza de mantequilla, 1 taza de azúcar morena...",0.999771,905.33,47.17,103.64,30.36,0.7
228,Alitas de pollo de mango-hábana,"30 secciones de ala de pollo, 1 (12 onzas) pue...",0.999683,1180.54,63.42,117.24,44.38,0.3
151,Pera clásica crujiente,"1 taza de avena enrollada, ? taza de azúcar mo...",0.999645,1168.70,59.99,128.41,40.78,0.3
315,"Ensalada de nectarina, tomate y pepino de verano","2 tomates, cortados en trozos, ½ pepino: pelad...",0.998769,924.76,35.87,152.20,18.98,0.3
21,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
1,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
11,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
31,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
295,Barras de granola sin avena y sin gluten (alim...,"1 ¼ tazas de almendras, divididas, 1 ¼ tazas d...",0.997911,1104.44,68.71,84.78,47.10,0.3
136,El mejor zapatero de melocotón,"1 (29 onzas) CAN Peaches en rodajas, 2 cuchara...",0.997309,1223.85,66.62,119.65,46.05,0.3


In [7]:
df3 = df2[df2['puntaje_platillo']>=0.5]
df3

,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo
46,Pastel de manzana de sartén de hierro de la ab...,"½ taza de mantequilla, 1 taza de azúcar morena...",0.999771,905.33,47.17,103.64,30.36,0.7
21,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
1,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
11,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7
31,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7


In [8]:
df4 = recomendador128.Calcular_Precios(df3, verbose=False)
df4

1/1 [==============================] - 0s 176ms/step


c:\Users\jsaur\Dropbox\UNIR MAESTRIA\MAESTRIA IA\SEMINARIO DE INNOVACION\repo\COMSINT\recomendaciones_comsint.py:1858: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFiltrados['costo_receta'] = Precios


,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo,costo_receta
46,Pastel de manzana de sartén de hierro de la ab...,"½ taza de mantequilla, 1 taza de azúcar morena...",0.999771,905.33,47.17,103.64,30.36,0.7,616.17
21,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7,810.24
1,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7,810.24
11,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7,810.24
31,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7,810.24


In [9]:
df4[df4['costo_receta']>=50].sort_values(by=['costo_receta','kcal', 'similitud'], ascending=True)

,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo,costo_receta
46,Pastel de manzana de sartén de hierro de la ab...,"½ taza de mantequilla, 1 taza de azúcar morena...",0.999771,905.33,47.17,103.64,30.36,0.7,616.17
21,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7,810.24
1,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7,810.24
11,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7,810.24
31,Relleno de pastel de manzana,"18 tazas de manzanas en rodajas finas, 3 cucha...",0.998038,1028.78,41.87,154.51,25.05,0.7,810.24


In [10]:
resultados = df4.to_dict(orient='records')

In [11]:
resultados[0]

{'nombre_del_platillo': 'Pastel de manzana de sartén de hierro de la abuela',
 'ingredientes': '½ taza de mantequilla, 1 taza de azúcar morena, 5 manzanas Granny Smith: peladas, con núcleo, descuidada y cortada en rodajas, 3 (9 pulgadas) costras de pastel preroleadas refrigeradas, 1 taza de azúcar blanca, dividida, 2 cucharaditas de canela molida, dividida, ¼ taza de azúcar blanco, 1 cucharada de mantequilla, cortado en trozos pequeños',
 'similitud': 0.999770641147656,
 'kcal': 905.33,
 'proteinas_gr': 47.17,
 'carbohidratos_gr': 103.64,
 'grasas_gr': 30.36,
 'puntaje_platillo': 0.7,
 'costo_receta': 616.17}